# Load the Teams
We are now going to load every team over every year that the league has existed.

This will create `teams.csv`.

In [7]:
import os
import requests
import pandas as pd

from dotenv import load_dotenv

load_dotenv()
league_id = os.getenv('league_id')
current_year = os.getenv('current_year')
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/{}".format(current_year, league_id)
history_url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}".format(league_id)
cookies = {
        "swid": os.getenv('swid'),
        "espn_s2": os.getenv('espn_s2')
    }

In [8]:
def get_data(args):
    season_id, load_url = args
    team_req = requests.get(load_url,
        cookies=cookies,
        params={"seasonId": season_id, "view": "mTeam"})
    teams = team_req.json()[0]["teams"]
    teams_df = pd.DataFrame(json_normalize(teams))
    teams_df["year"] = season_id
    return teams_df


current = requests.get(url,
    cookies=cookies,
    params={"view": "mTeam"})
curr_teams = current.json()["teams"]
year = current.json()["seasonId"]
prev = current.json()["status"]["previousSeasons"]
full_teams_df = pd.DataFrame(pd.json_normalize(curr_teams))
full_teams_df["year"] = year


In [9]:
loaded = map(get_data, list(map(lambda y: (y, history_url), prev)))
full_teams_df = full_teams_df.append(pd.concat(list(loaded)))
#full_teams_df.drop([c for c in full_teams_df.columns 
#    if c.startswith("draftStrategy") or c.startswith("record.away") or c.startswith("record.division") or c.startswith("record.home")
#       or c.startswith("valuesByStat") or c.startswith("transactionCounter.matchupAcquisitionTotals")], 
#    axis=1, inplace=True)
#full_teams_df.drop(columns=["isActive", "logoType", "owners", "watchList"], inplace=True)
full_teams_df.to_csv("teams.csv")

/tmp/ipykernel_673/2597704606.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  teams_df = pd.DataFrame(json_normalize(teams))
